<a href="https://colab.research.google.com/github/Shady91-ML/Loan_status-ML/blob/main/Loan_status_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score


#Data loading

In [2]:
loan_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project 5 - Loan status/train_u6lujuX_CVtuZ9i (1).csv')
loan_df = loan_df.copy()

In [3]:
loan_df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


#Data analysis

In [4]:
#Number of rows and columns
loan_df.shape

(614, 13)

In [5]:
#Statistical:
loan_df.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [6]:
#Missing values:
loan_df.isna().sum()

,0
Loan_ID,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14


In [7]:
#Dropping missing values only from categorical columns:
categorical_columns = loan_df.select_dtypes(include='object').columns
loan_df = loan_df.dropna(subset = categorical_columns)

In [8]:
#Filling missing numerical values with mean:
numerical_columns = loan_df.select_dtypes(include='number').columns
for col in numerical_columns:
    loan_df.loc[:, col] = loan_df[col].fillna(loan_df[col].mean())

loan_df_clean = loan_df.copy()

In [9]:
#Types of data
loan_df.dtypes

,0
Loan_ID,object
Gender,object
Married,object
Dependents,object
Education,object
Self_Employed,object
ApplicantIncome,int64
CoapplicantIncome,float64
LoanAmount,float64
Loan_Amount_Term,float64


In [10]:
#Label encoding:
loan_df_clean['Loan_Status'] = loan_df['Loan_Status'].map({'N':0,'Y':1})

In [11]:
loan_df_clean.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,144.917757,360.0,1.0,Urban,1
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,1


In [12]:
#Dependets column:
loan_df['Dependents'].value_counts()

,count
Dependents,
0,319
1,95
2,94
3+,46


In [13]:
#Dependents encoding:
dependents_map={'0':0,
                '1':1,
                '2':2,
                '3+':3}
loan_df_clean['Dependents']=loan_df['Dependents'].map(dependents_map)
loan_df_clean['Dependents'].value_counts()

,count
Dependents,
0,319
1,95
2,94
3,46


In [14]:
#Checking column:
print(loan_df['Gender'].value_counts() ,'\n',
  loan_df['Married'].value_counts(),'\n',
  loan_df['Education'].value_counts(),'\n',
  loan_df['Self_Employed'].value_counts(),'\n',
  loan_df['Property_Area'].value_counts()     )

Gender
Male      453
Female    101
Name: count, dtype: int64 
 Married
Yes    362
No     192
Name: count, dtype: int64 
 Education
Graduate        431
Not Graduate    123
Name: count, dtype: int64 
 Self_Employed
No     478
Yes     76
Name: count, dtype: int64 
 Property_Area
Semiurban    211
Urban        180
Rural        163
Name: count, dtype: int64


In [15]:
#Mapping values:
gender = {'Male' : 1,
          'Female' : 0}
married = {'Yes' : 1,
          'No' : 0}
education = {'Graduate' : 1,
          'Not Graduate' : 0}
employment = {'Yes' : 1,
            'No' : 0}
area = {'Urban' : 2,
        'Semiurban' : 1,
        'Rural' : 0}
loan_df_clean['Gender']=loan_df['Gender'].map(gender)
loan_df_clean['Married']=loan_df['Married'].map(married)
loan_df_clean['Education']=loan_df['Education'].map(education)
loan_df_clean['Self_Employed']=loan_df['Self_Employed'].map(employment)
loan_df_clean['Property_Area']=loan_df['Property_Area'].map(area)


In [16]:
#Types of data in clean dataset
loan_df_clean.dtypes

,0
Loan_ID,object
Gender,int64
Married,int64
Dependents,int64
Education,int64
Self_Employed,int64
ApplicantIncome,int64
CoapplicantIncome,float64
LoanAmount,float64
Loan_Amount_Term,float64


In [17]:
#Dropping first column - not important for ML:
loan_df_clean.drop('Loan_ID', axis=1, inplace=True)

In [18]:
loan_df_clean.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,1,0,5849,0.0,144.917757,360.0,1.0,2,1
1,1,1,1,1,0,4583,1508.0,128.000000,360.0,1.0,0,0
2,1,1,0,1,1,3000,0.0,66.000000,360.0,1.0,2,1
3,1,1,0,0,0,2583,2358.0,120.000000,360.0,1.0,2,1
4,1,0,0,1,0,6000,0.0,141.000000,360.0,1.0,2,1


#Spliting data on test and train data


In [26]:
X = loan_df_clean.drop('Loan_Status', axis=1)
y = loan_df_clean['Loan_Status']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1968)

#Train model

In [29]:
#Initializing model:
model = svm.SVC(kernel='linear')

In [30]:
#Train model:
model.fit(X_train,y_train)

SVC(kernel='linear')

#Model evaluation

In [31]:
#Accuracy score on train data:
train_predict = model.predict(X_train)
acc_score_train = accuracy_score(train_predict, y_train)
print('Acc score for train data: ', acc_score_train)

#Accuracy score on test data:
test_predict = model.predict(X_test)
acc_score_test = accuracy_score(test_predict, y_test)
print('Acc score for test data: ', acc_score_test)

Acc score for train data:  0.8058690744920993
Acc score for test data:  0.7837837837837838
